In [1]:
import pandas as pd
import numpy as np
import datetime

import psycopg2
import pymysql

from datetime import timedelta

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
host = '157.245.248.249'
port = int(5432)
dbname = 'ubuntu'
user = 'ubuntuuser'
password = '97cD8$pt@'


def get_query_results_postgres():
    with psycopg2.connect(host = host,
                          port = port,
                          database = dbname,
                          user = user,
                          password = password) as conn:
        sql = "SELECT * FROM bloomlive.loans_fact_table_materialized_summary_view"
        df = pd.read_sql(sql, conn)
        
    return df

    conn.close()

In [ ]:
loans = get_query_results_postgres()

loans.head(2)

/opt/tljh/user/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
loans = loans[loans['bloom_version'] == 2]

loans = loans[(loans['loan_status'] == 300) | (loans['loan_status'] == 600)| (loans['loan_status'] == 700)]

loans.shape

In [ ]:
host = '167.71.47.216'
port = int(3306)
user = 'tableau_reports'
password = 'change@123'
dbname = 'mifostenant-safaricom'

def get_query_results_mysql():
    with pymysql.connect(host = host,
                         port = port,
                         database = dbname,
                         user = user,
                         password = password) as conn:
        sql = "SELECT * FROM m_loan_transaction"
        df = pd.read_sql(sql, conn)
        
    return df

    conn.close()

In [ ]:
df_transactions = get_query_results_mysql()

df_transactions.head(2)

In [ ]:
# filtering df to only contain repayments
df_transactions = df_transactions[(df_transactions['transaction_type_enum'] == 2) & (df_transactions['is_reversed'] == 0)]

df_transactions.shape

In [ ]:
df_transactions.head(2)

In [ ]:
#renaming loan_id to loan_mifos_id
df_transactions.rename(columns = {'loan_id':'loan_mifos_id'}, inplace = True)

# grouping loan_mifos_id to find the max transaction date
transactions = df_transactions.groupby(['loan_mifos_id'], as_index=False)['transaction_date'].max()
transactions = transactions.rename(columns={'transaction_date':'max_transaction_date'})

transactions.head(2)

In [ ]:
transactions.shape

In [ ]:
#changing the dates column to datetime datatype
loans['disbursed_on_date'] = pd.to_datetime(loans['disbursed_on_date'])

loans['expected_matured_on_date'] = pd.to_datetime(loans['expected_matured_on_date'])

#changing the dates column to datetime datatype
transactions['max_transaction_date'] = pd.to_datetime(transactions['max_transaction_date'])

In [ ]:
transactions.shape

In [ ]:
# merging df_transactions with defaulters
df = pd.merge(loans, transactions, on = "loan_mifos_id", how = "left")
df.head(2)

In [ ]:
df.shape

In [ ]:
#create due_date fixed column to ensure due date tallies with the term frequency for each loan

df["due_date_fixed"] = df["disbursed_on_date"] + pd.to_timedelta(df["term_frequency"], unit='d')

df['due_date_fixed'] = pd.to_datetime(df['due_date_fixed'], errors = 'coerce')

df.head(2)

In [ ]:
df['curr_time'] = '2022-08-31'

df['curr_time'] = pd.to_datetime(df['curr_time'], errors = 'coerce')

df.head()

In [ ]:
def calc_days_past_due(df):
        
    res = 0
    loan_status = df['loan_status']
    expected_matured_on_date = df['expected_matured_on_date']
    max_transaction_date = df['max_transaction_date']
    curr_time = df['curr_time']
    
    if (loan_status == 600) and (max_transaction_date > expected_matured_on_date):
        res = (max_transaction_date - expected_matured_on_date).days
    elif (loan_status == 700) and (max_transaction_date > expected_matured_on_date):
        res = (max_transaction_date - expected_matured_on_date).days
    elif (loan_status == 300) and (curr_time > expected_matured_on_date):
        res = (curr_time - expected_matured_on_date).days
        
    return res if res > 0 else res

In [ ]:
df['days_past_due'] = df.apply(lambda x: calc_days_past_due(x), axis = 1)

df.head(2)

In [ ]:
def calc_end_rollover_date(df):
    tenure = df['repay_every']
    expected_matured_on_date = df['expected_matured_on_date']
    
    if tenure == 21:
        res = expected_matured_on_date + pd.to_timedelta(5, unit='d')
    elif tenure == 7:
        res = expected_matured_on_date + pd.to_timedelta(3, unit='d')
    elif tenure == 1:
        res = expected_matured_on_date + pd.to_timedelta(1, unit='d')
        
    return res

In [ ]:
df['end_rollover_date'] = df.apply(lambda x: calc_end_rollover_date(x), axis = 1)

df.head(2)

In [ ]:
#create days past due date column to ensure due date tallies with the term frequency for each loan

df["dpd_90_date"] = df["end_rollover_date"] + pd.to_timedelta(90, unit='d')

df['dpd_90_date'] = pd.to_datetime(df['dpd_90_date'])

df.head(2)

In [ ]:
df = df[['loan_mifos_id', 'store_number', 'term_frequency', 'principal_disbursed', 'curr_time', 'disbursed_on_date', 'total_expected_repayment', 'total_repayment', 'due_date_fixed', 'expected_matured_on_date', 'max_transaction_date', 'due_date_fixed', 'end_rollover_date', 'dpd_90_date', 'days_past_due']]

df.head(2)

In [ ]:
df_transactions = df_transactions[['loan_mifos_id', 'transaction_date', 'amount']]

df_transactions.head(2)

In [ ]:
df2 = pd.merge(df, df_transactions, on = 'loan_mifos_id', how = 'left')

df2.head(2)

In [ ]:
df2.info()

In [ ]:
df2['transaction_date'] = pd.to_datetime(df2['transaction_date'], errors='coerce')

df2['disbursed_on_date'] = pd.to_datetime(df2['disbursed_on_date'], errors='coerce')

df2['expected_matured_on_date'] = pd.to_datetime(df2['expected_matured_on_date'], errors='coerce')

df2['end_rollover_date'] = pd.to_datetime(df2['end_rollover_date'], errors='coerce')

df2['dpd_90_date'] = pd.to_datetime(df2['dpd_90_date'], errors='coerce')

df2['curr_time'] = pd.to_datetime(df2['curr_time'], errors='coerce')

In [ ]:
def calc_trans_by_due_date(df):
    
    expected_matured_on_date = df['expected_matured_on_date']
    transaction_date = df['transaction_date']
    
    if transaction_date <= expected_matured_on_date:
        return 'yes'
    else:
        return 'no'
    
    
def calc_trans_by_end_rollover_date(df):
    
    end_rollover_date = df['end_rollover_date']
    transaction_date = df['transaction_date']
    
    if transaction_date <= end_rollover_date:
        return 'yes'
    else:
        return 'no'
    
    
def calc_trans_by_dpd_90_date(df):
    
    dpd_90_date = df['dpd_90_date']
    transaction_date = df['transaction_date']
    
    if transaction_date <= dpd_90_date:
        return 'yes'
    else:
        return 'no'

In [ ]:
df2['transaction_by_due_date'] = df2.apply(lambda x: calc_trans_by_due_date(x), axis = 1)
df2['transaction_by_end_rollover_date'] = df2.apply(lambda x: calc_trans_by_end_rollover_date(x), axis = 1)
df2['transaction_by_dpd_90_date'] = df2.apply(lambda x: calc_trans_by_dpd_90_date(x), axis = 1)


df2.head(2)

In [ ]:
df2.shape

In [ ]:
trans_by_due_date = df2[df2['transaction_by_due_date'] == 'yes']

# grouping loan_mifos_id to find amount transacted by due date
amount_trans_by_due_date = trans_by_due_date.groupby(['loan_mifos_id'], as_index=False)['amount'].sum()
amount_trans_by_due_date = amount_trans_by_due_date.rename(columns={'amount':'amount_trans_by_due_date'})

amount_trans_by_due_date.head(2)

In [ ]:
amount_trans_by_due_date.shape

In [ ]:
transaction_by_end_rollover_date = df2[df2['transaction_by_end_rollover_date'] == 'yes']

# grouping loan_mifos_id to find amount transacted by due date
amount_transaction_by_end_rollover_date = transaction_by_end_rollover_date.groupby(['loan_mifos_id'], as_index=False)['amount'].sum()
amount_transaction_by_end_rollover_date = amount_transaction_by_end_rollover_date.rename(columns={'amount':'amount_transaction_by_end_rollover_date'})

amount_transaction_by_end_rollover_date.head(2)

In [ ]:
amount_transaction_by_end_rollover_date.shape

In [ ]:
transaction_by_dpd_90_date = df2[df2['transaction_by_dpd_90_date'] == 'yes']

# grouping loan_mifos_id to find amount transacted by due date
amount_transaction_by_dpd_90_date = transaction_by_dpd_90_date.groupby(['loan_mifos_id'], as_index=False)['amount'].sum()
amount_transaction_by_dpd_90_date = amount_transaction_by_dpd_90_date.rename(columns={'amount':'amount_transaction_by_dpd_90_date'})

amount_transaction_by_dpd_90_date.head(2)

In [ ]:
amount_transaction_by_dpd_90_date.shape

In [ ]:
final_df = pd.merge(amount_trans_by_due_date, amount_transaction_by_end_rollover_date, on = 'loan_mifos_id', how = 'outer')
final_df = pd.merge(final_df, amount_transaction_by_dpd_90_date, on = 'loan_mifos_id', how = 'outer')
final_df = final_df.fillna(0)

final_df.head(2)

In [ ]:
df_loans = df[['loan_mifos_id', 'store_number', 'term_frequency', 'principal_disbursed', 'curr_time', 'disbursed_on_date', 'expected_matured_on_date', 'end_rollover_date', 'dpd_90_date', 'total_expected_repayment']]
final_df = pd.merge(df_loans, final_df, on = 'loan_mifos_id', how = 'left')
final_df = final_df.fillna(0)

final_df.head(2)

In [ ]:
final_df.shape

In [ ]:
total_principal_disbursed = final_df.groupby(['store_number'], as_index=False)['principal_disbursed'].sum()
total_principal_disbursed = total_principal_disbursed.rename(columns={'principal_disbursed':'total_principal_disbursed'})

total_principal_disbursed.head()

In [ ]:
total_principal_disbursed.shape

In [ ]:
total_repayments_by_due_date = final_df.groupby(['store_number'], as_index=False)['amount_trans_by_due_date'].sum()
total_repayments_by_due_date = total_repayments_by_due_date.rename(columns={'amount_trans_by_due_date':'total_repayments_by_due_date'})

total_repayments_by_due_date.head()

In [ ]:
total_repayments_by_due_date.shape

In [ ]:
principal_repayments_by_due_date = pd.merge(total_principal_disbursed, total_repayments_by_due_date, on = 'store_number', how = 'outer')

principal_repayments_by_due_date.head()

In [ ]:
principal_repayments_by_due_date['repayments_vs_principal'] = principal_repayments_by_due_date['total_repayments_by_due_date'] / principal_repayments_by_due_date['total_principal_disbursed']

principal_repayments_by_due_date.head(10)

In [ ]:
principal_repayments_by_due_date_95 = principal_repayments_by_due_date[principal_repayments_by_due_date['repayments_vs_principal'] >= 0.95]

principal_repayments_by_due_date_95.head(10)

In [ ]:
principal_repayments_by_due_date_95.shape

In [ ]:
#get list of all target store numbers

principal_repayments_by_due_date_95_list = list(principal_repayments_by_due_date_95["store_number"].unique())
len(principal_repayments_by_due_date_95_list)

In [ ]:
principal_repayments_by_due_date_95.info()

In [ ]:
previous_limits = pd.read_excel("/home/jupyter-ngenia-jackline/Limits_refresh_summary_20220922_multiple_products_1.xlsx")
#previous_limits = previous_limits[['store_number', 'final_21_limit', 'final_7_limit', 'final_1_limit']]
previous_limits = previous_limits.rename(columns={'final_21_limit':'bf_final_21_limit', 'final_7_limit':'bf_final_7_limit', 'final_1_limit':'bf_final_1_limit'})

previous_limits[['bf_final_21_limit', 'bf_final_7_limit', 'bf_final_1_limit']].sum()

In [ ]:
new_limits = pd.read_excel("/home/jupyter-ngenia-jackline/final_limits_new.xlsx")
new_limits = new_limits[['store_number', 'final_21_limit', 'final_7_limit', 'final_1_limit']]
new_limits['final_21_limit'] = new_limits['final_21_limit'].fillna(0)
new_limits = new_limits.rename(columns={'final_21_limit':'af_final_21_limit', 'final_7_limit':'af_final_7_limit', 'final_1_limit':'af_final_1_limit'})

new_limits[['af_final_21_limit', 'af_final_7_limit', 'af_final_1_limit']].sum()

In [ ]:
all_limits = pd.merge(previous_limits, new_limits, on = 'store_number', how = 'outer')

all_limits.head()

In [ ]:
all_limits[['bf_final_21_limit', 'bf_final_7_limit', 'bf_final_1_limit', 'af_final_21_limit', 'af_final_7_limit', 'af_final_1_limit']].sum()

In [ ]:
all_limits['store_number'] = all_limits['store_number'].astype(str)

In [ ]:
zeroized = all_limits[(all_limits['bf_final_21_limit'] > 0) & (all_limits['af_final_21_limit'] == 0)]

zeroized.head()

In [ ]:
zeroized['store_number'] = zeroized['store_number'].astype(str)

zeroized.info()

In [ ]:
zeroized.shape

In [ ]:
target_customers_scope = zeroized.loc[zeroized["store_number"].isin(principal_repayments_by_due_date_95_list)]

print(target_customers_scope.shape)

In [ ]:
target_customers_scope.head()

In [ ]:
target_customers_scope[['bf_final_21_limit', 'af_final_21_limit']].sum()

In [ ]:
target_customers_scope = target_customers_scope[['store_number', 'bf_final_21_limit', 'bf_final_7_limit', 'bf_final_1_limit']]
target_customers_scope = target_customers_scope.rename(columns={'bf_final_21_limit':'af95_final_21_limit', 'bf_final_7_limit':'af95_final_7_limit', 'bf_final_1_limit':'af95_final_1_limit'})

target_customers_scope.head()

In [ ]:
comb_limits = pd.merge(all_limits, target_customers_scope, on = 'store_number', how = 'outer')
comb_limits = comb_limits.fillna(0)

comb_limits.head()

In [ ]:
comb_limits[['bf_final_21_limit', 'bf_final_7_limit', 'bf_final_1_limit', 'af_final_21_limit', 'af_final_7_limit', 'af_final_1_limit', 'af95_final_21_limit', 'af95_final_7_limit', 'af95_final_1_limit']].sum()

In [ ]:
def calc_final_21_limits(df):
    
    bf_final_21_limit = df['bf_final_21_limit']
    af_final_21_limit = df['af_final_21_limit']
    af95_final_21_limit = df['af95_final_21_limit']
    
    if (af_final_21_limit == 0) and (af95_final_21_limit > 0):
        return af95_final_21_limit
    elif (af_final_21_limit > 0) and (af95_final_21_limit == 0):
        return af_final_21_limit
    elif (af_final_21_limit == 0) and (af95_final_21_limit == 0):
        return af_final_21_limit

In [ ]:
comb_limits['final_21_limits'] = comb_limits.apply(lambda x: calc_final_21_limits(x), axis=1)

comb_limits.head(10)

In [ ]:
comb_limits[['bf_final_21_limit', 'bf_final_7_limit', 'bf_final_1_limit', 'af_final_21_limit', 'af_final_7_limit', 'af_final_1_limit', 'af95_final_21_limit', 'af95_final_7_limit', 'af95_final_1_limit', 'final_21_limits']].sum()

In [ ]:
#comb_limits = comb_limits[['store_number', 'final_21_limits', 'bf_final_7_limit', 'bf_final_1_limit']]

comb_limits[['final_21_limits', 'bf_final_7_limit', 'bf_final_1_limit']].sum()

In [ ]:
comb_limits.to_excel('comb_limits_new.xlsx')